In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup

# Google スプレッドシートの認証情報ファイル（JSON）のパス
credentials_file = 'credentials.json'

# Google スプレッドシートのキー
spreadsheet_key = '1CvuclRh4f3GGDP42uQSrtEHOWn-V4F9LV_Iiy7RNnRQ'

# ワークシート名
worksheet_name = '小学校_詳細'

def write_data_to_sheet(data):
    # 認証情報の読み込み
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, ['https://www.googleapis.com/auth/spreadsheets'])

    # Google スプレッドシートに接続
    client = gspread.authorize(credentials)

    # スプレッドシートを開く
    spreadsheet = client.open_by_key(spreadsheet_key)

    # ワークシートを取得
    worksheet = spreadsheet.worksheet(worksheet_name)

    # データをリスト形式に変換
    values = []
    for row in data:
        name = row['学校名']
        address = row['所在地']
        phone = row['電話番号']
        website = row['公式HP']
        uniform = row['制服']
        meal = row['給食']
        event = row['行事']
        tuition = row['学費']
        selection = row['選考の有無']
        selection_method = row['選考方法']
        values.append([name, address, phone, website, uniform, meal, event, tuition, selection, selection_method])

    # 書き込む範囲の開始セルを決定
    start_cell = len(worksheet.get_all_values()) + 1

    # データを一括で書き込む
    worksheet.update(f'A{start_cell}', values)

def scrape_school_data(url):
    # HTMLを取得
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # 基本情報テーブルを取得
    basic_info_table = soup.find('table', class_='table-binfo')
    rows = basic_info_table.find_all('tr')
    
    # 受験情報テーブルを取得
    jinfo_table = soup.find('table', class_='table-jinfo')

    # 学校情報を格納するリスト
    school_data = []
    
     # 初期化
    uniform = ''
    meal = ''
    event = ''
    tuition = ''
    selection = ''
    selection_method = ''

    # 学校名を取得
    school_name_element = soup.find('h1', class_='mod-school-name')
    school_name = school_name_element.text.strip() if school_name_element else ''

    # 所在地を取得
    address_element = soup.find('p', class_='tx-address')
    address = address_element.text.strip() if address_element else ''

    # 電話番号を取得
    phone_element = soup.find('th', class_='data-tel').find_next('td')
    phone = phone_element.text.strip() if phone_element else ''

    # 公式HPを取得
    website_element = soup.find('th', class_='data-website').find_next('p')
    website = website_element.text.strip()  if website_element else ''

    # 各行を処理
    for row in rows:
        header = row.find('th').text.strip()
        value = row.find('td').text.strip()
        
        # 制服の情報を取得
        if header == '制服':
            uniform = value
        
        # 給食の情報を取得
        elif header == '給食':
            meal = value
        
        # 行事の情報を取得
        elif header == '行事':
            event = value
        
        # 学費の情報を取得
        elif header == '学費':
            tuition = value
            
    # 受験情報テーブルが存在する場合のみ処理
    if jinfo_table:
        jinfo_rows = jinfo_table.find_all('tr')
        
        # 各行を処理（受験情報テーブル）
        for row in jinfo_rows:
            header = row.find('th').text.strip()
            value = row.find('td').text.strip()
            
            # 選考の有無の情報を取得
            if header == '選考の有無':
                selection = value
                
            # 選考方法の情報を取得
            elif header == '選考方法':
                selection_method = value
    
    # 学校情報を辞書として追加
    school_data.append({
        '学校名': school_name,
        '所在地': address,
        '電話番号': phone,
        '公式HP': website,
        '制服': uniform,
        '給食': meal,
        '行事': event,
        '学費': tuition,
        '選考の有無': selection,
        '選考方法': selection_method
    })

    return school_data

# スクレイピング対象のURLリスト
urls = [
    'https://www.minkou.jp//primary/school/31678/',
    'https://www.minkou.jp//primary/school/31677/',
    'https://www.minkou.jp//primary/school/31674/',
    'https://www.minkou.jp//primary/school/31676/',
    'https://www.minkou.jp//primary/school/31688/',
    'https://www.minkou.jp//primary/school/31686/',
    'https://www.minkou.jp//primary/school/31692/',
    'https://www.minkou.jp//primary/school/31689/',
    'https://www.minkou.jp//primary/school/31681/',
    'https://www.minkou.jp//primary/school/31679/',
    'https://www.minkou.jp//primary/school/31687/',
    'https://www.minkou.jp//primary/school/31693/',
    'https://www.minkou.jp//primary/school/31685/',
    'https://www.minkou.jp//primary/school/31673/',
    'https://www.minkou.jp//primary/school/31675/',
    'https://www.minkou.jp//primary/school/31680/',
    'https://www.minkou.jp//primary/school/31672/',
    'https://www.minkou.jp//primary/school/31683/',
    'https://www.minkou.jp//primary/school/31684/',
    'https://www.minkou.jp//primary/school/31690/',
    'https://www.minkou.jp//primary/school/31691/',
    'https://www.minkou.jp//primary/school/31682/'
]

# 情報を格納するリスト
data = []

# 各URLから情報を取得
for url in urls:
    school_data = scrape_school_data(url)
    data.extend(school_data)

# データをGoogle スプレッドシートに書き込む
write_data_to_sheet(data)

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup

# Google スプレッドシートの認証情報ファイル（JSON）のパス
credentials_file = 'credentials.json'

# Google スプレッドシートのキー
spreadsheet_key = '1CvuclRh4f3GGDP42uQSrtEHOWn-V4F9LV_Iiy7RNnRQ'

# ワークシート名
worksheet_name = '小学校_評価'

def write_data_to_sheet(data):
    # 認証情報の読み込み
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, ['https://www.googleapis.com/auth/spreadsheets'])

    # Google スプレッドシートに接続
    client = gspread.authorize(credentials)

    # スプレッドシートを開く
    spreadsheet = client.open_by_key(spreadsheet_key)

    # ワークシートを取得
    worksheet = spreadsheet.worksheet(worksheet_name)

    # データをリスト形式に変換
    values = []
    for row in data:
        name = row['名称']
        rating = row['評価']
        icons = row['アイコン'].split(', ')
        url = row['URL']
        values.append([name, rating] + icons + [url])

    # 書き込む範囲の開始セルを決定
    start_cell = len(worksheet.get_all_values()) + 1

    # データを一括で書き込む
    worksheet.update(f'A{start_cell}', values)

# スクレイピング対象のURL
url = 'https://www.minkou.jp/primary/search/kanagawa/14118/'

# HTMLを取得
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

# 情報を格納するリスト
data = []

# 2ページの情報を取得
for page in range(1, 3):
    # URLを構築
    url_with_page = f'https://www.minkou.jp/primary/search/kanagawa/14118/page={page}/'

    # HTMLを取得
    r = requests.get(url_with_page)
    soup = BeautifulSoup(r.content, 'html.parser')

    # sch-search-list要素を取得
    search_list = soup.find('div', class_='sch-search-list')

    # 各sch-searchBox要素を処理
    for search_box in search_list.find_all('div', class_='sch-searchBox'):
        # 各要素から情報を抽出
        name = search_box.find('h3').text.strip()
        rating = search_box.find('div', class_='review-txt-pointTotal').text.strip()
        icon_elements = search_box.find_all('li')
        icons = [icon.text.strip() for icon in icon_elements]
        icons_str = ', '.join(icons)  # アイコンを文字列に変換
        url = search_box.find('a')['href']

        # 抽出した情報を辞書として追加
        data.append({
            '名称': name,
            '評価': rating,
            'アイコン': icons_str,
            'URL': 'https://www.minkou.jp/'+url
        })

# データを Google スプレッドシートに書き込む
write_data_to_sheet(data)

In [4]:
print(url)

/primary/school/31682/
